# TF IDF

In [ ]:
docA = "The cat sat on my face"
docB = "The dog sat on my bed"

In [ ]:
bowA = docA.split(" ")
bowB = docB.split(" ")

In [ ]:
bowB

In [ ]:
wordSet = set(bowA).union(set(bowB))

In [ ]:
wordSet

In [ ]:
wordDictA = dict.fromkeys(wordSet, 0) 
wordDictB = dict.fromkeys(wordSet, 0)

In [ ]:
wordDictA

In [ ]:
for word in bowA:
    wordDictA[word]+=1
    
for word in bowB:
    wordDictB[word]+=1

In [ ]:
wordDictA

In [ ]:
import pandas as pd
pd.DataFrame([wordDictA, wordDictB])

In [ ]:
def computeTF(wordDict, bow):
    tfDict = {}
    bowCount = len(bow)
    for word, count in wordDict.items():
        tfDict[word] = count/float(bowCount)
    return tfDict

In [ ]:
tfBowA = computeTF(wordDictA, bowA)
tfBowB = computeTF(wordDictB, bowB)

In [ ]:
tfBowA

In [ ]:
tfBowB

In [ ]:
def computeIDF(docList):
    import math
    idfDict = {}
    N = len(docList)
    
    idfDict = dict.fromkeys(docList[0].keys(), 0)
    for doc in docList:
        for word, val in doc.items():
            if val > 0:
                idfDict[word] += 1
    
    for word, val in idfDict.items():
        idfDict[word] = math.log10(N / float(val))
        
    return idfDict

In [ ]:
idfs = computeIDF([wordDictA, wordDictB])

In [ ]:
def computeTFIDF(tfBow, idfs):
    tfidf = {}
    for word, val in tfBow.items():
        tfidf[word] = val*idfs[word]
    return tfidf

In [ ]:
tfidfBowA = computeTFIDF(tfBowA, idfs)
tfidfBowB = computeTFIDF(tfBowB, idfs)

In [ ]:
import pandas as pd
pd.DataFrame([tfidfBowA, tfidfBowB])

In [ ]:
from sklearn.feature_selection import 

# Sentence embeddings

In [ ]:
import numpy as np
from sklearn.decomposition import PCA
from typing import List

In [ ]:
class Word:
    def __init__(self, text, vector):
        self.text = text
        self.vector = vector
    
    def __str__(self):
        return self.text + ' : ' + str(self.vector)

    def __str__(self):
        return self.text + ' : ' + str(self.vector)


In [ ]:
class Sentence:
    def __init__(self, word_list):
        self.word_list = word_list

    def len(self):
        return len(self.word_list)

    def __str__(self):
        word_str_list = [word.text for word in self.word_list]
        return ' '.join(word_str_list)

    def __repr__(self):
        return self.__str__()


In [ ]:
def get_word_frequency(word_text):
    return 0.0001

In [ ]:
def sentence_to_vec(sentence_list, embedding_size, a):
    sentence_set = []
    for sentence in sentence_list:
        vs = np.zeros(embedding_size)
        sentence_length = sentence.len()
        for word in sentence.word_list:
            a_value = a / (a + get_word_frequency(word.text))  # smooth inverse frequency, SIF
            vs = np.add(vs, np.multiply(a_value, word.vector)) # vs += sif * word_vector
        
        vs = np.divide(vs, sentence_length)  # weighted average
        sentence_set.append(vs)  # add to our existing re-calculated set of sentences
    
    # calculate PCA of this sentence set
    pca = PCA()
    pca.fit(np.array(sentence_set))
    u = pca.components_[0]  # the PCA vector
    u = np.multiply(u, np.transpose(u))  # u x uT
    
    if len(u) < embedding_size:
        for i in range(embedding_size - len(u)):
            u = np.append(u, 0)  # add needed extension for multiplication below
    
    # resulting sentence vectors, vs = vs -u x uT x vs
    sentence_vecs = []
    for vs in sentence_set:
        sub = np.multiply(u,vs)
        sentence_vecs.append(np.subtract(vs, sub))

    return sentence_vecs

# textrank

In [1]:
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt') # one time execution
import re

[nltk_data] Error loading punkt: <urlopen error [Errno 61] Connection
[nltk_data]     refused>


In [2]:
df = pd.read_csv('/Users/zn-nlp/Documents/tennis_articles_v4.csv', encoding='utf-8')

In [3]:
df


,article_id,article_text,source
0,1,Maria Sharapova has basically no friends as te...,https://www.tennisworldusa.org/tennis/news/Mar...
1,2,"BASEL, Switzerland (AP), Roger Federer advance...",http://www.tennis.com/pro-game/2018/10/copil-s...
2,3,Roger Federer has revealed that organisers of ...,https://scroll.in/field/899938/tennis-roger-fe...
3,4,Kei Nishikori will try to end his long losing ...,http://www.tennis.com/pro-game/2018/10/nishiko...
4,5,"Federer, 37, first broke through on tour over ...",https://www.express.co.uk/sport/tennis/1036101...
5,6,Nadal has not played tennis since he was force...,https://www.express.co.uk/sport/tennis/1037119...
6,7,"Tennis giveth, and tennis taketh away. The end...",http://www.tennis.com/pro-game/2018/10/tennisc...
7,8,Federer won the Swiss Indoors last week by bea...,https://www.express.co.uk/sport/tennis/1038186...


In [4]:
sentences = []
for s in df['article_text']:
    sentences.append(sent_tokenize(s))

In [5]:
sentences = [y for x in sentences for y in x]

In [6]:
# remove punctuations, numbers and special characters
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]

In [7]:
nltk.download('stopwords')# one time execution

[nltk_data] Error loading stopwords: <urlopen error [Errno 61]
[nltk_data]     Connection refused>


False

In [8]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [9]:
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

In [10]:
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

In [ ]:
# download pretrained GloVe word embeddings
! wget http://nlp.stanford.edu/data/glove.6B.zip

In [ ]:
! unzip glove*.zip

In [11]:
# Extract word vectors
word_embeddings = {}
f = open('/Users/zn-nlp/Documents/glove.6B.50d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [12]:
sentence_vectors = []
for i in clean_sentences:
    if len(i) != 0:
        v = sum([word_embeddings.get(w, np.zeros((50,))) for w in i.split()])/(len(i.split())+0.001)
    else:
        v = np.zeros((50,))
    sentence_vectors.append(v)

In [13]:
len(sentence_vectors)


119

In [14]:
# similarity matrix
sim_mat = np.zeros([len(sentences), len(sentences)])

In [15]:
from sklearn.metrics.pairwise import cosine_similarity

In [16]:
for i in range(len(sentences)):
    for j in range(len(sentences)):
        if i != j:
            sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,50), sentence_vectors[j].reshape(1,50))[0,0]

In [17]:
import networkx as nx

nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

In [18]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

In [19]:
# Specify number of sentences to form the summary
sn = 3

# Generate summary
for i in range(sn):
    print(ranked_sentences[i][1])

When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match.
Speaking at the Swiss Indoors tournament where he will play in Sundays final against Romanian qualifier Marius Copil, the world number three said that given the impossibly short time frame to make a decision, he opted out of any commitment.
Major players feel that a big event in late November combined with one in January before the Australian Open will mean too much tennis and too little rest.
